## Question 2

In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
def build_mu_sigma():
    """We build mu and Sigma."""
    mu = np.array([
        3.66, 1.33, 2.51, 2.24, 2.62, 2.03,
        1.91, 1.39, 2.61, 3.58, 5.23, 8.36
    ], dtype=float)

    Sigma = np.array([
        [ 73.73,  40.79,  39.46,  38.01,  49.40,  58.51,  46.08,  42.26,  -1.22,  12.44,  -2.31, 131.17],
        [ 40.79,  39.54,  32.15,  34.43,  34.25,  36.88,  33.11,  26.50,  -3.13,  19.06,   0.53, 136.78],
        [ 39.46,  32.15,  54.98,  31.62,  33.80,  41.20,  26.48,  29.76,  -2.90,  25.34,  -3.62, 151.82],
        [ 38.01,  34.43,  31.62,  39.61,  35.73,  33.64,  38.12,  26.00,  -0.08,  13.23,  -5.61,  73.96],
        [ 49.40,  34.25,  33.80,  35.73,  68.10,  36.88,  37.02,  31.64,  14.42,   3.33,   1.43,  -0.12],
        [ 58.51,  36.88,  41.20,  33.64,  36.88,  66.93,  40.89,  39.08, -13.71,  22.07, -14.96, 210.26],
        [ 46.08,  33.11,  26.48,  38.12,  37.02,  40.89,  75.40,  38.58,  19.01,  19.37,  30.95,  52.63],
        [ 42.26,  26.50,  29.76,  26.00,  31.64,  39.08,  38.58,  47.35,  -3.53,  17.02,  -8.28, 111.37],
        [ -1.22,  -3.13,  -2.90,  -0.08,  14.42, -13.71,  19.01,  -3.53, 242.03, -90.65, 196.18,-507.15],
        [ 12.44,  19.06,  25.34,  13.23,   3.33,  22.07,  19.37,  17.02, -90.65, 110.87, -50.97, 360.36],
        [ -2.31,   0.53,  -3.62,  -5.61,   1.43, -14.96,  30.95,  -8.28, 196.18, -50.97, 428.23,-504.38],
        [131.17, 136.78, 151.82,  73.96,  -0.12, 210.26,  52.63, 111.37,-507.15, 360.36,-504.38,2863.07]
    ], dtype=float)

    return mu, Sigma


def build_reduced_quadratic(mu, Sigma, phi=5.0):
    """
    We build Q, q, r such that Je(x) = x^T Q x + q^T x + r,
    where x in R^(n-1) and x_n = 1 - sum_{i=1}^{n-1} x_i.
    """
    n = mu.shape[0]
    m = n - 1
    e = np.ones(m)

    A = Sigma[:m, :m]
    b = Sigma[:m, m]
    c = Sigma[m, m]

    Q = A - np.outer(b, e) - np.outer(e, b) + c * np.outer(e, e)
    q = 2.0 * b - 2.0 * c * e - phi * (mu[:m] - mu[m] * e)
    r = c - phi * mu[m]

    return Q, q, r


In [3]:
def Je(x, Q, q, r):
    """It is the reduced solution Je(x) = x^T Q x + q^T x + r."""
    x = np.asarray(x, dtype=float)
    return float(x @ Q @ x + q @ x + r)


def grad_Je(x, Q, q):
    """It is the gradient of Je: ∇Je(x) = 2Qx + q."""
    x = np.asarray(x, dtype=float)
    return 2.0 * (Q @ x) + q

In [4]:
def line_search_strong_wolfe(f, grad, xk, pk, c1=1e-4, c2=0.9, alpha1=1.0, alpha_max=1e6, max_iter=50):
    """
    Strong Wolfe line search for phi(alpha)=f(xk+alpha pk).
    It uses a bisection-based zoom procedure.
    """
    def phi(a):
        return f(xk + a * pk)

    def dphi(a):
        return float(grad(xk + a * pk) @ pk)

    phi0 = phi(0.0)
    dphi0 = dphi(0.0)
    if dphi0 >= 0.0:
        # pk is not a descent direction
        return 0.0, False

    alpha_prev = 0.0
    phi_prev = phi0
    alpha = alpha1

    def zoom(alow, ahigh, phi_low):
        for _ in range(60):
            a = 0.5 * (alow + ahigh)  # bisection
            phi_a = phi(a)

            if (phi_a > phi0 + c1 * a * dphi0) or (phi_a >= phi_low):
                ahigh = a
            else:
                dphi_a = dphi(a)
                if abs(dphi_a) <= -c2 * dphi0:
                    return a, True
                if dphi_a * (ahigh - alow) >= 0:
                    ahigh = alow
                alow = a
                phi_low = phi_a

            if abs(ahigh - alow) < 1e-14:
                return a, False

        return a, False

    for i in range(max_iter):
        phi_a = phi(alpha)

        if (phi_a > phi0 + c1 * alpha * dphi0) or (i > 0 and phi_a >= phi_prev):
            return zoom(alpha_prev, alpha, phi_prev)

        dphi_a = dphi(alpha)
        if abs(dphi_a) <= -c2 * dphi0:
            return alpha, True

        if dphi_a >= 0:
            return zoom(alpha, alpha_prev, phi_a)

        alpha_prev = alpha
        phi_prev = phi_a
        alpha = min(2.0 * alpha, alpha_max)

    return alpha, False

In [5]:
def steepest_descent_wolfe(Q, q, r, x0, tol=1e-5, max_iter=200000, c1=1e-4, c2=0.9):
    """
    Steepest descent:
        p_k = -∇Je(x_k)
        x_{k+1} = x_k + alpha_k p_k
    with alpha_k satisfying the strong Wolfe conditions.
    """
    x = np.array(x0, dtype=float)

    for k in range(max_iter):
        gk = grad_Je(x, Q, q)
        if np.linalg.norm(gk) <= tol:
            return x, k, Je(x, Q, q, r), True

        pk = -gk

        f = lambda z: Je(z, Q, q, r)
        g = lambda z: grad_Je(z, Q, q)

        alpha, ok = line_search_strong_wolfe(f, g, x, pk, c1=c1, c2=c2)

        # We perform a fallback which is the exact step for a quadratic to ensure progress
        if not ok:
            denom = pk @ (2.0 * (Q @ pk))
            alpha = (gk @ gk) / denom

        x = x + alpha * pk

    return x, max_iter, Je(x, Q, q, r), False

In [6]:

phi = 5.0
mu, Sigma = build_mu_sigma()
Q, q, r = build_reduced_quadratic(mu, Sigma, phi=phi)

# Starting point, we'll keep the same starting point throughout the project
x0 = np.zeros(11)

x_star, n_iter, Je_star, converged = steepest_descent_wolfe(Q, q, r, x0, tol=1e-5)

print("Converged:", converged)
print("Starting point x0 =", x0)
print("Iterations =", n_iter)
print("Optimal reduced x* =", x_star)
print("Je(x*) =", Je_star)

Converged: True
Starting point x0 = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Iterations = 55150
Optimal reduced x* = [ 0.2685102  -0.6015868  -0.18651315  0.95014298  0.05797521 -0.01420054
 -0.39572757  0.18019928  0.24183311  0.41863451  0.06426102]
Je(x*) = 3.0412849729891605


## Question 3
We'll reuse what we've already coded in 2 question and add the definition for the Hessian matrix of Je, we'll code when wolfe strong is satisfied, redefine the line search strong wolfe and we'll define the Newton-Wolfe method

In [7]:
def hess_Je(Q):
    """The hessian matrix of Je: ∇^2Je(x) = 2Q."""
    return 2.0 * Q

In [8]:
def wolfe_strong_satisfied(f, grad, xk, pk, alpha, c1=1e-4, c2=0.9):
    """
    Check strong Wolfe conditions for phi(alpha)=f(xk+alpha pk).
    Returns True if both Armijo and strong curvature are satisfied.
    """
    phi0 = f(xk)
    g0 = grad(xk)
    dphi0 = float(g0 @ pk)

    # pk must be a descent direction for Wolfe
    if dphi0 >= 0.0:
        return False

    x_new = xk + alpha * pk
    phi_a = f(x_new)
    ga = grad(x_new)
    dphi_a = float(ga @ pk)

    armijo = (phi_a <= phi0 + c1 * alpha * dphi0)
    curvature = (abs(dphi_a) <= -c2 * dphi0)
    return armijo and curvature

In [9]:
def line_search_strong_wolfe(f, grad, xk, pk, c1=1e-4, c2=0.9, alpha1=1.0, alpha_max=1e6, max_iter=50):
    """
    Strong Wolfe line search.
    Uses bracketing then a bisection zoom.
    """
    phi0 = f(xk)
    g0 = grad(xk)
    dphi0 = float(g0 @ pk)
    if dphi0 >= 0.0:
        return 0.0, False

    def phi(a):
        return f(xk + a * pk)

    def dphi(a):
        return float(grad(xk + a * pk) @ pk)

    alpha_prev = 0.0
    phi_prev = phi0
    alpha = alpha1

    # Bracketing phase
    for i in range(max_iter):
        phi_a = phi(alpha)

        if (phi_a > phi0 + c1 * alpha * dphi0) or (i > 0 and phi_a >= phi_prev):
            a_lo, a_hi = alpha_prev, alpha
            break

        dphi_a = dphi(alpha)
        if abs(dphi_a) <= -c2 * dphi0:
            return alpha, True

        if dphi_a >= 0.0:
            a_lo, a_hi = alpha, alpha_prev
            break

        alpha_prev = alpha
        phi_prev = phi_a
        alpha = min(2.0 * alpha, alpha_max)
    else:
        return alpha, False

    # Ensure ordering for zoom
    a_lo, a_hi = (min(a_lo, a_hi), max(a_lo, a_hi))
    phi_lo = phi(a_lo)

    # Zoom phase
    for _ in range(60):
        a = 0.5 * (a_lo + a_hi)
        phi_a = phi(a)

        if (phi_a > phi0 + c1 * a * dphi0) or (phi_a >= phi_lo):
            a_hi = a
        else:
            dphi_a = dphi(a)
            if abs(dphi_a) <= -c2 * dphi0:
                return a, True
            if dphi_a * (a_hi - a_lo) >= 0.0:
                a_hi = a_lo
            a_lo = a
            phi_lo = phi_a

        if abs(a_hi - a_lo) < 1e-14:
            return a, False

    return a, False

In [10]:
def newton_wolfe(Q, q, r, x0, tol=1e-8, max_iter=50, c1=1e-4, c2=0.9):
    """
    Newton's method with strong Wolfe step lengths.

    Key robustness features:
    - Try alpha=1 first and accept it if it satisfies strong Wolfe.
    - Check convergence both before and after the update.
    """
    x = np.array(x0, dtype=float)

    # Symmetrize Q to reduce numerical asymmetry (good practice)
    Qs = 0.5 * (Q + Q.T)
    H = 2.0 * Qs  # constant Hessian

    f = lambda z: Je(z, Qs, q, r)
    g = lambda z: grad_Je(z, Qs, q)

    for k in range(max_iter):
        grad_k = g(x)
        if np.linalg.norm(grad_k) <= tol:
            return x, k, f(x), True

        # Newton direction: solve H p = -grad
        pk = np.linalg.solve(H, -grad_k)

        # We first try full Newton step
        alpha = 1.0
        if not wolfe_strong_satisfied(f, g, x, pk, alpha, c1=c1, c2=c2):
            # Otherwise we run Wolfe line search
            alpha, ok = line_search_strong_wolfe(f, g, x, pk, c1=c1, c2=c2, alpha1=1.0)
            if not ok:
                # Safe fallback
                alpha = 1.0

        x = x + alpha * pk

        # Check convergence right after the update
        if np.linalg.norm(g(x)) <= tol:
            return x, k + 1, f(x), True

    return x, max_iter, f(x), False

In [11]:
phi = 5.0
mu, Sigma = build_mu_sigma()
Q, q, r = build_reduced_quadratic(mu, Sigma, phi=phi)

# We take the same starting point as in Q2
x0 = np.zeros(11)

x_star, n_iter, Je_star, converged = newton_wolfe(Q, q, r, x0)

print("Converged:", converged)
print("Starting point x0 =", x0)
print("Iterations =", n_iter)
print("Optimal reduced x* =", x_star)
print("Je(x*) =", Je_star)
3.0412849729891605

Converged: True
Starting point x0 = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Iterations = 1
Optimal reduced x* = [ 0.26851035 -0.60158785 -0.18651334  0.95014403  0.05797526 -0.01420058
 -0.39572781  0.18019933  0.24183317  0.41863455  0.06426108]
Je(x*) = 3.041284972984613


3.0412849729891605

## Question 4


In [12]:
def conjugate_gradient(A, b, x0=None, tol=1e-10, max_iter=None):
    """
    Linear Conjugate Gradient to solve A x = b for SPD matrix A.
    Stops when ||r_k|| <= tol where r_k = b - A x_k.
    """
    n = b.size
    x = np.zeros(n, dtype=float) if x0 is None else np.array(x0, dtype=float)

    r = b - A @ x
    p = r.copy()
    rs_old = float(r @ r)

    if max_iter is None:
        max_iter = n  # theoretical upper bound

    for k in range(max_iter):
        Ap = A @ p
        alpha = rs_old / float(p @ Ap)
        x = x + alpha * p
        r = r - alpha * Ap

        rs_new = float(r @ r)
        if np.sqrt(rs_new) <= tol:
            return x, k + 1, np.sqrt(rs_new), True

        beta = rs_new / rs_old
        p = r + beta * p
        rs_old = rs_new

    return x, max_iter, np.sqrt(rs_old), False

In [13]:
phi = 5.0
mu, Sigma = build_mu_sigma()
Q, q, r = build_reduced_quadratic(mu, Sigma, phi=phi)

# Minimizer solves (2Q) x = -q
H = 2.0 * Q
b = -q

x0 = np.zeros(11)
x_star, n_iter, res_norm, converged = conjugate_gradient(H, b, x0=x0, tol=1e-10, max_iter=200)

print("Converged:", converged)
print("Iterations:", n_iter)
print("Residual norm ||b - Hx||:", res_norm)
print("x* (reduced):", x_star)
print("Je(x*):", Je(x_star, Q, q, r))


Converged: True
Iterations: 16
Residual norm ||b - Hx||: 8.574691791826074e-11
x* (reduced): [ 0.26851035 -0.60158785 -0.18651334  0.95014403  0.05797526 -0.01420058
 -0.39572781  0.18019933  0.24183317  0.41863455  0.06426108]
Je(x*): 3.0412849729832487


## Question 5

In [14]:
def solve_with_scipy(Q, q, r, x0):
    """
    Solve min Je(x) using scipy.optimize.minimize with BFGS.
    We provide the analytic gradient to speed up convergence and improve accuracy.
    """

    fun = lambda x: Je(x, Q, q, r)
    jac = lambda x: grad_Je(x, Q, q)

    # We'll use Scipy since it has BFGS implemented in it and it's a quasi-Newton method
    res = minimize(
        fun=fun,
        x0=x0,
        jac=jac,
        method="BFGS",
        options={
            "gtol": 1e-10,      # stop when ||grad|| is small
            "maxiter": 10_000,
            "disp": False
        }
    )
    return res
def newton_exact_quadratic(Q, q):
    """
    For Je(x)=x^T Q x + q^T x + r, the minimizer satisfies 2Qx + q = 0,
    hence x* = -0.5 Q^{-1} q (assuming Q is invertible / SPD in practice).
    """
    return np.linalg.solve(2.0 * Q, -q)

In [15]:
phi = 5.0
mu, Sigma = build_mu_sigma()
Q, q, r = build_reduced_quadratic(mu, Sigma, phi=phi)

# We use the same starting point as in previous questions
x0 = np.zeros(11)

# The solution with scipy
res = solve_with_scipy(Q, q, r, x0)
x_scipy = res.x
Je_scipy = Je(x_scipy, Q, q, r)
grad_norm_scipy = np.linalg.norm(grad_Je(x_scipy, Q, q))

# The solution we got, as a comparison, from question 3
x_newton, it_newton, Je_newton, ok_newton = newton_wolfe(Q, q, r, x0, tol=1e-10)
grad_norm_newton = np.linalg.norm(grad_Je(x_newton, Q, q))

# We'll use these metrics to compare BFGS and the one we used in question 3
diff_x = np.linalg.norm(x_scipy - x_newton)
diff_J = abs(Je_scipy - Je_newton)

print("---- SciPy (BFGS) ----")
print("success:", res.success)
print("iterations (nit):", res.nit, "| nfev:", res.nfev, "| njev:", res.njev)
print("Je(x*) =", Je_scipy)
print("||grad|| =", grad_norm_scipy)

print("\n---- Our Newton+Wolfe (Q3) ----")
print("converged:", ok_newton)
print("iterations:", it_newton)
print("Je(x*) =", Je_newton)
print("||grad|| =", grad_norm_newton)

print("\n---- Comparison ----")
print("||x_scipy - x_newton|| =", diff_x)
print("|Je_scipy - Je_newton| =", diff_J)

---- SciPy (BFGS) ----
success: True
iterations (nit): 16 | nfev: 21 | njev: 21
Je(x*) = 3.0412849729841582
||grad|| = 5.903314256149596e-11

---- Our Newton+Wolfe (Q3) ----
converged: True
iterations: 1
Je(x*) = 3.041284972984613
||grad|| = 2.227797943108433e-12

---- Comparison ----
||x_scipy - x_newton|| = 6.390720527027872e-13
|Je_scipy - Je_newton| = 4.547473508864641e-13


## Question 6
Now we once again consider the constrains in order to solve the optimization problem. We'll go back to the full 12-weight vector for simpler computations.

In [16]:
def objective_full(w, Sigma, mu, phi=5.0):
    """Objective J(w) = w^T Sigma w - phi * mu^T w (full 12D variable on the simplex)."""
    return float(w @ Sigma @ w - phi * (mu @ w))


def grad_full(w, Sigma, mu, phi=5.0):
    """Gradient of J: ∇J(w) = 2 Sigma w - phi mu."""
    return 2.0 * (Sigma @ w) - phi * mu

def project_to_simplex(v):
    """
    Euclidean projection onto the simplex {w: w >= 0, sum(w) = 1}.
    Algorithm: sort v, find threshold theta, then w = max(v - theta, 0).
    """
    v = np.asarray(v, dtype=float)
    n = v.size

    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    rho = np.where(u - (cssv - 1.0) / (np.arange(n) + 1) > 0)[0][-1]
    theta = (cssv[rho] - 1.0) / (rho + 1)

    w = np.maximum(v - theta, 0.0)
    return w

In [17]:
def projected_gradient_descent_simplex(Sigma, mu, phi=5.0, w0=None, tol=1e-8, max_iter=200000):
    """
    Projected Gradient Descent on the simplex with Armijo backtracking.
    We backtrack alpha until Armijo decrease holds along the feasible direction d.
    """
    n = mu.size
    if w0 is None:
        w = np.ones(n) / n
    else:
        w = project_to_simplex(w0)

    f = lambda x: objective_full(x, Sigma, mu, phi)
    g = lambda x: grad_full(x, Sigma, mu, phi)

    for k in range(max_iter):
        grad = g(w)
        # Projected gradient mapping
        w_proj = project_to_simplex(w - grad)
        pg_norm = np.linalg.norm(w - w_proj)

        if pg_norm <= tol:
            return w, k, f(w), True

        # We perform a Armijo backtracking on the projected step
        alpha = 1.0
        c1 = 1e-4
        f_w = f(w)

        while True:
            w_new = project_to_simplex(w - alpha * grad)
            d = w_new - w

            # If the projection gives almost no movement, we consider that we are numerically stationary
            if np.linalg.norm(d) < 1e-14:
                return w, k, f_w, True

            # Armijo sufficient decrease using directional derivative of the grad^T d
            if f(w_new) <= f_w + c1 * (grad @ d):
                break

            alpha *= 0.5
            if alpha < 1e-16:
                # If the step became too small, we stop safely
                return w, k, f_w, False

        w = w_new

    return w, max_iter, f(w), False

In [18]:
phi = 5.0
mu, Sigma = build_mu_sigma()

w_star, n_iter, J_star, ok = projected_gradient_descent_simplex(
    Sigma=Sigma,
    mu=mu,
    phi=phi,
    w0=np.ones(12) / 12,
    tol=1e-8,
    max_iter=200000
)

print("Converged:", ok)
print("Iterations:", n_iter)
print("w* =", w_star)
print("sum(w*) =", w_star.sum(), "min(w*) =", w_star.min())
print("J(w*) =", J_star)

# Reduced variable x in R^11
x_star = w_star[:11]
print("x* (reduced) =", x_star)
print("constraint 1 - sum(x*) =", 1.0 - x_star.sum())

Converged: True
Iterations: 275
w* = [0.09546618 0.         0.         0.27193496 0.03241073 0.
 0.         0.03515613 0.20256739 0.34348564 0.01897897 0.        ]
sum(w*) = 0.9999999999999987 min(w*) = 0.0
J(w*) = 7.918837100790903
x* (reduced) = [0.09546618 0.         0.         0.27193496 0.03241073 0.
 0.         0.03515613 0.20256739 0.34348564 0.01897897]
constraint 1 - sum(x*) = 1.3322676295501878e-15
